# Data Preprocessing And Feature Engineering

In [ ]:
# loda training datasets
import pandas as pd
df = pd.read_csv("./data/datasets/train_qWM28Yl.csv")
df.head()

: 

In [ ]:
df.describe()

In [ ]:
# df.info()

# identify important features for training model with.

In [ ]:
# identify important columns.
# engine_type, model, make, population_density, age_of_policyholder, age_of_car, area_cluster, fuel_type, max_torque, max_power, airbags, is_parking_sensors, is_parking_camera, cylinder, transmission_type, gear_box, length, width, height, gross_weight, is_speed_alert, is_claim

# features = [ 'engine_type', 'model', 'make', 'population_density', 'age_of_policyholder', 'age_of_car', 'area_cluster', 'fuel_type', 'max_torque', 'max_power', 'airbags', 'is_parking_sensors', 'is_parking_camera', 'cylinder', 'transmission_type', 'gear_box', 'length', 'width', 'height', 'gross_weight', 'is_speed_alert', "is_claim" ]

print( "features in dataset:\n".upper(), df.columns )

features = [ 'engine_type', 'fuel_type', 'population_density', 'cylinder', 'transmission_type', 'gear_box', 'length', 'width', 'height', 'gross_weight', 'is_speed_alert', "is_claim" ]
print( "\nSelected Features:\n".upper(), features)

In [ ]:
df = df[features]
# assume there is no duplicates or NA values or rows.
# df.dropna(inplace=True, axis=0 )
# df.drop_duplicates(inplace=True,)
df.head()

In [ ]:
# bhp = torque * rpm / 5252
print("data shape:", df.shape )

##### remove max torque and power from features, until an optimal idea is found.

In [ ]:
# # preprocess max_torque and max_power features. | remove them
# import re
# text = "91Nm@4250rpm"
# text = str(text).lower()
# text = text.split("@")
# text[0] = re.sub("nm", '', text[0])
# text[1] = re.sub("rpm", '', text[1])
# text
    

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# ss = StandardScaler()
# a = [[4], [5], [7], [3]]
# scaler = ss.fit( a )
# r = scaler.transform(a)
# # rr = [ i[0] for i in r]
# i = scaler.inverse_transform(r)
# print("Norm\n", r)
# print("Inverse\n", i)

In [ ]:
# try inverse transform, policu holder age
# dd = df['age_of_policyholder'][ : 5]
# dd = [ [d,] for d in dd ]
# dd
# scaler.inverse_transform(dd)

# Class to transform data, normalise, standardize and scale features.

In [ ]:
from sklearn.preprocessing import LabelEncoder, LabelEncoder, StandardScaler

_standardScalerObj = StandardScaler()

class Preprocss():
    '''preprocess features and scale them.'''
    def __init__(self):
        super().__init__()
        
    def __str__(self) -> str:
        return '''preprocess features and scale them.'''
        
    def transmission_type_norm(self, value ):
        '''0 -> automatic | 1 -> manuel'''
        value = str(value).lower()
        if 'automatic' == value: return 0
        elif 'manual' == value: return 1
        else: return 3
        
    def fuel_type_norm(self, value ):
        '''0 -> petrol | 1 ->diesel | 2 -> cng | 3 -> unknown '''
        value = str(value).lower()
        if value == 'petrol': return 0
        elif value == 'diesel': return 1
        elif value == 'cng': return 2
        else: return 3
    
    def speed_alert(self, value):
        '''0 -> No | 1 -> Yes'''
        value = str(value).lower()
        if value == 'no': return 0
        elif value == 'yes': return 1
        else: return 0
    
    def engine( self, value ):
        '''I can't use labelencoder from scikit-learn. its not optimal'''
        '''['F8D Petrol Engine', '1.2 L K12N Dualjet', '1.0 SCe',
       '1.5 L U2 CRDi', '1.5 Turbocharged Revotorq', 'K Series Dual jet',
       '1.2 L K Series Engine', 'K10C', 'i-DTEC', 'G12B',
       '1.5 Turbocharged Revotron']'''
        if value != None:
            value = str(value).lower()
            if value == 'f8d petrol engine'.lower(): return 0
            elif value == '1.2 L K12N Dualjet'.lower(): return 1
            elif value == '1.0 SCe'.lower(): return 2
            elif value == '1.5 L U2 CRDi'.lower(): return 3
            elif value == '1.5 Turbocharged Revotorq'.lower(): return 4
            elif value == 'K Series Dual jet'.lower(): return 5
            elif value == '1.2 L K Series Engine'.lower(): return 6
            elif value == 'K10C'.lower(): return 7
            elif value == 'i-DTEC'.lower(): return 8
            elif value == 'G12B'.lower(): return 9
            elif value == '1.5 Turbocharged Revotron'.lower(): return 10
        else: return value
        
    def standard_scaler(self, value ):
        '''use standard scaler to scal the age of the car'''
        array = [[value],]
        _standardScalerObj.fit(array)
        res = _standardScalerObj.transform(array)
        return res[0][0]
    
        

In [ ]:
preproObj = Preprocss()
df['engine_type'] = df['engine_type'].apply( preproObj.engine )
df['is_speed_alert'] = df['is_speed_alert'].apply( preproObj.speed_alert )
df['fuel_type'] = df['fuel_type'].apply( preproObj.fuel_type_norm )
df['transmission_type'] = df['transmission_type'].apply( preproObj.transmission_type_norm )
df.head()

# Apply Imblearn to balance classes in dataset.

In [ ]:
# visualise class distributions before over sampling.
import seaborn as sns
import matplotlib.pyplot as plt

plt.title("Class Distribution Before Undersampling")
sns.countplot(data=df, x="is_claim")
plt.xlabel("Classes")

In [ ]:
# import tomeklink and create object with strategy of reducing from majority class. 
from imblearn.under_sampling import TomekLinks

# tomekObj = TomekLinks(sampling_strategy='auto')
tomekObj = TomekLinks()

# get target and feature data in variables.
target = df['is_claim']
features = df[['engine_type','fuel_type','population_density','cylinder','transmission_type','gear_box','length','width','height','gross_weight','is_speed_alert']]
# 
print( f"Data shape: {features.shape}" )
print(f"target distribution before undersampling \n{target.value_counts()}" )

In [ ]:
# 
for i in range(10):
    x_sample, y_sample = tomekObj.fit_resample(features, target)
# 
print(f"Data shape: {x_sample.shape}" )
print(f"target distribution after undersampling \n{y_sample.value_counts()}" )

#### Save new data to file.

In [ ]:
df.to_csv("./data/processed_data.csv", index=False)